In [1]:
import os 
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x) 

# import missingno as msno
# import matplotlib.pyplot as plt
# import seaborn as sns 
# import warnings
# warnings.filterwarnings("ignore")

In [10]:
os.chdir('C:\\Users\\rhian\\Downloads\\CYPLAN 255\\AP-Exams\\Data')
ap = pd.read_csv('ap_model.csv')
ap

,year,ncessch,enrollment,AP_oneormore_relative,teachers_per_student,security_gaurds_per_students,avg_teacher_salary,%Asian,%Black,%Hispanic,%Indigenous \nAmerican,%Multiracial,%Pacific \nIslander,%White,%Female,%Male,enrl_AP,enrl_AP_science,enrl_AP_math,enrl_AP_other,enrl_gifted_talented,students_SAT_ACT,students_chronically_absent,students_report_harass_dis,students_susp_in_sch,students_susp_out_sch_single,students_susp_out_sch_multiple,students_arrested
0,2011,60000103278.0000,83.0000,-0.2329,0.0002,0.0000,17499.4056,0.0039,-0.0045,-0.0388,-0.0029,-0.0694,0.0004,0.1114,-0.0093,0.0093,0.0855,0.0317,-0.0140,-0.0548,0.0587,-0.0187,-0.0059,0.0000,-0.0108,0.0157,0.0311,0.0000
1,2011,60000310347.0000,22.2500,-0.2826,-0.0017,-0.0026,3164.3876,-0.0238,0.0080,-0.0452,0.0032,-0.0157,0.0031,0.0705,-0.0005,0.0005,0.0030,-0.0052,0.0095,0.0001,-0.0094,-0.0365,0.0140,0.0006,0.0043,0.0178,0.0234,0.0000
2,2011,60001302405.0000,-92.2500,3.6447,0.0004,-0.0003,-15597.9657,-0.0300,-0.0034,0.0053,0.0009,0.0037,0.0171,0.0063,0.0130,-0.0130,-0.0178,-0.0274,0.0175,0.0038,0.0027,0.0086,-0.0016,0.0000,0.0695,-0.0081,0.0381,0.0000
3,2011,60001311010.0000,-88.2500,1.7782,0.0011,-0.0003,-8989.7469,-0.0413,0.0013,-0.0035,0.0010,-0.0126,0.0266,0.0285,0.0102,-0.0102,-0.0043,-0.0070,-0.0213,-0.0355,-0.0092,0.0114,-0.0059,0.0000,0.0014,0.0137,0.0020,0.0008
4,2011,60001609086.0000,128.0000,-2.5556,0.0001,0.0003,1794.2415,-0.0155,-0.0074,-0.0499,0.0000,0.0033,0.0135,0.0561,-0.0004,0.0004,0.0027,-0.0110,0.0096,-0.0161,-0.0122,-0.0257,-0.0262,0.0000,-0.0046,0.0077,0.0019,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4456,2017,69113511047.0000,-162.0000,0.9247,-0.0059,-0.0008,17302.0010,-0.0020,-0.0268,0.1479,0.0000,-0.1070,-0.0034,-0.0087,-0.0024,0.0024,0.0293,-0.1378,-0.1205,0.0594,-0.0192,0.1071,-0.0452,0.0000,-0.0878,0.0117,-0.0353,-0.0011
4457,2017,69113512749.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4458,2017,69113513992.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4459,2017,69113603761.0000,-1.3333,-1.0904,0.0002,0.0000,-10080.8001,0.0001,0.0000,0.0333,-0.0018,-0.0139,0.0000,-0.0140,-0.0330,0.0330,0.0216,0.0028,-0.0045,0.0058,0.0060,0.0058,-0.0044,-0.0018,0.0000,-0.0232,0.0205,0.0000


In [4]:
from sklearn.feature_selection import SequentialFeatureSelector

In [21]:
from sklearn.linear_model import LinearRegression

In [43]:
X = np.array(ap.drop(['ncessch', 'AP_oneormore_relative'], axis=1)).reshape(4461, 26)
y = np.array(ap.AP_oneormore_relative)
lr = LinearRegression()

sfs = SequentialFeatureSelector(lr, n_features_to_select=3, 
                                direction='forward', scoring='neg_mean_squared_error', 
                                cv = 10)
sfs.fit(X, y)

best_cols = sfs.get_support()
best_cols = [i for i, x in enumerate(best_cols) if x]
ap.columns[best_cols]
# sfs.transform(X).shape
# (150, 3)

Index(['year', 'security_gaurds_per_students', '%Female'], dtype='object')

In [46]:
from sklearn.model_selection import KFold, GridSearchCV#, train_test_split

In [50]:
# Show how max_depth affects train and cross-validated accuracy
# model = DecisionTreeClassifier()  

cv = KFold(n_splits=10, shuffle=True, random_state=1)
# params = {'n_features_to_select':[4, 6, 8, 10, 12, 14, 16, 18, 20]}
params = {'n_features_to_select':[4, 6]}
cv_model = GridSearchCV(sfs, param_grid=params, scoring='neg_mean_squared_error', refit=True, 
                        return_train_score=True, cv=cv)
cv_model.fit(X, y) 
cv_results = pd.DataFrame(cv_model.cv_results_)
print(cv_model.best_estimator_)
cv_results[['param_n_features_to_select', 'mean_test_score']].sort_values(by = 'mean_test_score', 
                                                                          ascending = False) 

C:\Users\rhian\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\rhian\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "C:\Users\rhian\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 199, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true,
  File "C:\Users\rhian\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 236, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "C:\Users\rhian\anaconda3\lib\site-packages\sklearn\metrics\_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'SequentialFeatureSelector' object has no attribute 'predict'

  warnings.warn(
C:\Users\rhian\a

KeyboardInterrupt: 